<a href="https://colab.research.google.com/github/solobala/RMSL_9/blob/main/RS_lecture_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

In [ ]:
!wget https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
!unzip master.zip

--2023-06-23 16:55:35--  https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master [following]
--2023-06-23 16:55:35--  https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   1.06M  2.92MB/s    in 0.4s    

2023-06-23 16:55:36 (2.92 MB/s) - ‘master.zip’ saved [1111929]

Archive:  master.zip
dfe2a910caf170a1f0fd2174867169ce737c9dc7
   creating: netology-recsys-master/
   creating: netology-rec

In [ ]:
prefix = 'netology-recsys-master/lecture-1'

links = pd.read_csv(os.path.join(prefix, 'links.csv'))
movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))
tags = pd.read_csv(os.path.join(prefix, 'tags.csv'))

In [ ]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|')) # рАзбиваем строку жанров на отделные

In [ ]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [ ]:
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

In [ ]:
X_train_counts.todense()[0]

matrix([[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
count_vect.vocabulary_

{'adventure': 1,
 'animation': 2,
 'children': 3,
 'comedy': 4,
 'fantasy': 8,
 'romance': 15,
 'drama': 7,
 'action': 0,
 'crime': 5,
 'thriller': 17,
 'horror': 10,
 'mystery': 13,
 'scifi': 16,
 'war': 18,
 'musical': 12,
 'documentary': 6,
 'imax': 11,
 'western': 19,
 'filmnoir': 9,
 'nogenreslisted': 14}

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
np.square(np.array(X_train_tfidf.todense()[0] - X_train_tfidf.todense()[1])).sum()

0.3728445184397122

In [ ]:
neigh = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean')
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=7)

In [ ]:
test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [ ]:
res

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608]]),
 array([[6774, 9096, 3582,  863, 3576, 3376, 2302]]))

In [ ]:
movies.iloc[res[1][0]]

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
3582,4911,Jabberwocky (1977),Adventure|Comedy|Fantasy
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
3576,4899,Black Knight (2001),Adventure|Comedy|Fantasy
3376,4591,Erik the Viking (1989),Adventure|Comedy|Fantasy
2302,3052,Dogma (1999),Adventure|Comedy|Fantasy


In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
tags.head()

In [ ]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [ ]:
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [ ]:
movies_with_tags[movies_with_tags.title == 'Toy Story (1995)']

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09


In [ ]:
movies_with_tags.tag.unique()

In [ ]:
movies_with_tags.dropna(inplace=True)

In [ ]:
movies_with_tags.title.unique().shape

In [ ]:
tag_strings = []
movies = []

for movie, group in tqdm_notebook(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    movies.append(movie)

<ipython-input-33-b36be720bf58>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie, group in tqdm_notebook(movies_with_tags.groupby('title')):


  0%|          | 0/1572 [00:00<?, ?it/s]

In [ ]:
tag_strings[:5]

['artistic Funny humorous inspiring intelligent quirky romance ZooeyDeschanel',
 'lawyers',
 'creepy suspense',
 'Shakespearesortof',
 'dogs remake']

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(tag_strings)

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='manhattan')
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='manhattan', n_jobs=-1, n_neighbors=10)

In [ ]:
for i in range(len(movies)):
    if 'Magnolia (1999)' == movies[i]:
        print(i)

822


In [ ]:
tag_strings[822]

'L.A.'

In [ ]:
# test = change_string('pixar pixar fun')
test = change_string('pixar | pixar | fun')

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [ ]:
res

(array([[0.        , 0.53824232, 1.33670485, 1.33670485, 2.33670485,
         2.33670485, 2.33670485, 2.33670485, 2.33670485, 2.33670485]]),
 array([[1432,  211,  661,  822,    5,   11,    6,    9,    1,    8]]))

In [ ]:
for i in res[1][0]:
    print(movies[i])

Toy Story (1995)
Bug's Life, A (1998)
In a Lonely Place (1950)
Magnolia (1999)
101 Dalmatians (One Hundred and One Dalmatians) (1961)
21 Grams (2003)
11'09"01 - September 11 (2002)
13 Going on 30 (2004)
...And Justice for All (1979)
127 Hours (2010)


In [ ]:
import time as tm
t = 1687544867892
# last_update_time_struct = tm.localtime(tm.gmtime(float(t/1000.)))


In [ ]:
print(f'Last updated: {tm.strftime("%Y-%m-%d %H:%M:%S", tm.gmtime(float(t/1000.)))}')

Last updated: 2023-06-23 18:27:47


In [ ]:
child_results=dict()
results=dict()
child_results[1] = {'test_accuracy_score': 0.9894859813084113, 'test_precision_score': 0.9927884615384616, 'test_recall_score': 0.9856801909307876, 'test_f1_score': 0.9892215568862276, 'test_log_loss': 0.3789636454463254, 'test_roc_auc': 0.9894076011862176}
child_results[2] = {'test_accuracy_score': 0.9906542056074766, 'test_precision_score': 0.9904534606205251, 'test_recall_score': 0.9904534606205251, 'test_f1_score': 0.9904534606205251, 'test_log_loss': 0.33685657373006705, 'test_roc_auc': 0.990650071271361}
child_results[3] = {'test_accuracy_score': 0.991822429906542, 'test_precision_score': 0.9881516587677726, 'test_recall_score': 0.9952267303102625, 'test_f1_score': 0.9916765755053507, 'test_log_loss': 0.29474950201380873, 'test_roc_auc': 0.9918925413565044}
child_results[4] = {'test_accuracy_score': 0.9964953271028038, 'test_precision_score': 0.9976076555023924, 'test_recall_score': 0.9952267303102625, 'test_f1_score': 0.9964157706093191, 'test_log_loss': 0.12632121514877528, 'test_roc_auc': 0.996469200395406}
child_results[9] = {'test_accuracy_score': 0.9906432748538012, 'test_precision_score': 0.9880952380952381, 'test_recall_score': 0.992822966507177, 'test_f1_score': 0.9904534606205251, 'test_log_loss': 0.33725055802682735, 'test_roc_auc': 0.9906906594549615}

In [ ]:
for k, v in child_results.items():
  for key, value in v.items():
    results['cv_child_' + key] =0

for k, v in child_results.items():
  for key, value in v.items():
    results['cv_child_' + key] +=value


